In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from pandas import Series, DataFrame

import plotly
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sklearn
from sklearn.model_selection import TimeSeriesSplit
from sklearn import preprocessing

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import os
import json
import datetime

from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense

from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.utils import check_array
from sklearn.metrics import mean_squared_error as mse
from math import sqrt

from datetime import datetime

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving AAPL.csv to AAPL (1).csv
Saving GOOG.csv to GOOG (1).csv


In [ ]:
df = pd.read_csv('GOOG.csv', sep='\t')
df2 = pd.read_csv('AAPL.csv', sep='\t')

In [ ]:
data_google = df[['Date', 'Open', 'Low', 'High', 'Close']]
data_google['Date'] = pd.to_datetime(data_google['Date'], utc=False)
data_google= data_google.sort_values('Date')
data_google.to_csv("google_stock_prices.csv")
data_google.head()

,Date,Open,Low,High,Close
0,2010-01-04,312.304413,310.954468,313.579620,312.204773
1,2010-01-05,312.418976,309.609497,312.747742,310.829926
2,2010-01-06,311.761444,302.047852,311.761444,302.994293
3,2010-01-07,303.562164,295.218445,303.861053,295.940735
4,2010-01-08,294.894653,293.455048,300.498657,299.885956


In [ ]:
data_apple = df2[['Date', 'Open', 'Low', 'High', 'Close']]
data_apple['Date'] = pd.to_datetime(data_apple['Date'], utc=False)
data_apple = data_apple.sort_values('Date')
data_apple.to_csv("apple_stock_prices.csv")
data_apple.head()

,Date,Open,Low,High,Close
0,2010-01-04,30.490000,30.340000,30.642857,30.572857
1,2010-01-05,30.657143,30.464285,30.798571,30.625713
2,2010-01-06,30.625713,30.107143,30.747143,30.138571
3,2010-01-07,30.250000,29.864286,30.285715,30.082857
4,2010-01-08,30.042856,29.865715,30.285715,30.282858


In [ ]:
#extracting the columns we want 

# feature_google = data_google.columns[4]
# target_google = data_google.columns[0]

#Preprosessing Google Data
stock_data_g= data_google[['Date'] + ['Close']]
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaler_data = scaler.fit_transform(stock_data_g['Close'].values.reshape(-1, 1)).tolist()
scaler_data = pd.DataFrame(scaler_data, columns=['Close'])
stock_data_g = pd.DataFrame(pd.concat([scaler_data, stock_data_g['Date']], axis=1), columns=stock_data_g.columns)
stock_data_g.head(10)


,Date,Close
0,2010-01-04,0.090365
1,2010-01-05,0.089057
2,2010-01-06,0.081603
3,2010-01-07,0.074892
4,2010-01-08,0.078646
5,2010-01-11,0.078214
6,2010-01-12,0.073177
7,2010-01-13,0.071570
8,2010-01-14,0.072878
9,2010-01-15,0.068210


In [ ]:
# feature_apple = data_apple.columns[4]
# target_apple = data_apple.columns[0]

#Preprosessing Apple Data
stock_data_a = data_apple[['Date'] + ['Close']]
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaler_data = scaler.fit_transform(stock_data_a['Close'].values.reshape(-1, 1)).tolist()
scaler_data = pd.DataFrame(scaler_data, columns=['Close'])
stock_data_a = pd.DataFrame(pd.concat([scaler_data, stock_data_a['Date']], axis=1), columns=stock_data_a.columns)
stock_data_a.head(10)

,Date,Close
0,2010-01-04,0.015330
1,2010-01-05,0.015589
2,2010-01-06,0.013208
3,2010-01-07,0.012936
4,2010-01-08,0.013913
5,2010-01-11,0.012608
6,2010-01-12,0.010939
7,2010-01-13,0.012985
8,2010-01-14,0.012133
9,2010-01-15,0.009690


In [ ]:
# Let's plot the data
trace0 = go.Scatter(
    x=stock_data_g['Date'],
    y=stock_data_g['Close'],
    name='Google',
    line=dict(
        color='rgb(205, 12, 24)',
        width=2)
)

trace1 = go.Scatter(
    x=stock_data_a['Date'],
    y=stock_data_a['Close'],
    name='Apple',
    mode='lines',
    line=dict(
        color='rgb(22, 96, 167)',
        width=2)
)

fig =make_subplots(rows=2, cols=1)

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 2, 1)

fig['layout'].update(height=600, width=800, title='Google vs Apple Stock')
fig.show()

In [ ]:
def split_sequence(data, ts_col, x_col, n_steps=3, n_outs=1):
    #We want to create a 
    sequence = data[x_col].values
    ts = data.loc[n_steps:len(sequence)-n_outs, ts_col].values

    X, Y = [], []

    for i in range(len(sequence)):
        end_ix = i + n_steps

        # Extract the features and label
        if end_ix > (len(sequence) - n_outs):
            break
        seq_x = sequence[i: end_ix]
        
        #If we want more than one feature to be considered a target(2 for example) 
        # here we want the last feature of the sequence to be taken as target
        if n_outs > 1:
            seq_y = np.array(sequence[end_ix: end_ix + n_outs])
        else:
            seq_y = np.array(sequence[end_ix])
        
        X.append(seq_x)
        Y.append(seq_y)
        
    
    X = np.array(X)
    Y = np.array(Y)
    
    if n_outs < 2:
        Y = np.expand_dims(Y, -1)

    columns = ['%s (t-%d)' % (x_col, i) for i in range(n_steps, 0, -1)]

    if n_outs > 1:
        columns = columns + ['%s (t)' % (x_col) if i == 0 else '%s (t+%d)' % (x_col, i) for i in range(0, n_outs)]
    else:
        columns = columns + ['%s (t)' % (x_col)]


    _data = pd.DataFrame(np.concatenate([X, Y], axis=1), columns=columns)
    _data[ts_col] = ts

    _data = _data[[ts_col] + columns]
    _data = _data.dropna()
    _data = _data.reset_index(drop=True)


    return _data

In [ ]:
# Making Data Sequence of Apple
sequence_len = 30
n_outs = 1
feature_apple = data_apple.columns[4]
target_apple = data_apple.columns[0]
new_data_apple = split_sequence(stock_data_a, ts_col=target_apple , x_col=feature_apple, n_steps=sequence_len, n_outs=n_outs)
new_data_apple.head(10)

,Date,Close (t-30),Close (t-29),Close (t-28),Close (t-27),Close (t-26),Close (t-25),Close (t-24),Close (t-23),Close (t-22),...,Close (t-9),Close (t-8),Close (t-7),Close (t-6),Close (t-5),Close (t-4),Close (t-3),Close (t-2),Close (t-1),Close (t)
0,2010-02-17,0.015330,0.015589,0.013208,0.012936,0.013913,0.012608,0.010939,0.012985,0.012133,...,0.005012,0.000000,0.002381,0.001445,0.002890,0.002143,0.004621,0.005815,0.007924,0.007330
1,2010-02-18,0.015589,0.013208,0.012936,0.013913,0.012608,0.010939,0.012985,0.012133,0.009690,...,0.000000,0.002381,0.001445,0.002890,0.002143,0.004621,0.005815,0.007924,0.007330,0.007595
2,2010-02-19,0.013208,0.012936,0.013913,0.012608,0.010939,0.012985,0.012133,0.009690,0.016050,...,0.002381,0.001445,0.002890,0.002143,0.004621,0.005815,0.007924,0.007330,0.007595,0.006716
3,2010-02-22,0.012936,0.013913,0.012608,0.010939,0.012985,0.012133,0.009690,0.016050,0.013739,...,0.001445,0.002890,0.002143,0.004621,0.005815,0.007924,0.007330,0.007595,0.006716,0.005843
4,2010-02-23,0.013913,0.012608,0.010939,0.012985,0.012133,0.009690,0.016050,0.013739,0.011184,...,0.002890,0.002143,0.004621,0.005815,0.007924,0.007330,0.007595,0.006716,0.005843,0.003498
5,2010-02-24,0.012608,0.010939,0.012985,0.012133,0.009690,0.016050,0.013739,0.011184,0.003979,...,0.002143,0.004621,0.005815,0.007924,0.007330,0.007595,0.006716,0.005843,0.003498,0.006011
6,2010-02-25,0.010939,0.012985,0.012133,0.009690,0.016050,0.013739,0.011184,0.003979,0.007693,...,0.004621,0.005815,0.007924,0.007330,0.007595,0.006716,0.005843,0.003498,0.006011,0.006946
7,2010-02-26,0.012985,0.012133,0.009690,0.016050,0.013739,0.011184,0.003979,0.007693,0.009697,...,0.005815,0.007924,0.007330,0.007595,0.006716,0.005843,0.003498,0.006011,0.006946,0.008775
8,2010-03-01,0.012133,0.009690,0.016050,0.013739,0.011184,0.003979,0.007693,0.009697,0.011051,...,0.007924,0.007330,0.007595,0.006716,0.005843,0.003498,0.006011,0.006946,0.008775,0.011826
9,2010-03-02,0.009690,0.016050,0.013739,0.011184,0.003979,0.007693,0.009697,0.011051,0.005054,...,0.007330,0.007595,0.006716,0.005843,0.003498,0.006011,0.006946,0.008775,0.011826,0.011728


In [ ]:
# Making Data Sequence of Google

sequence_len = 30
n_outs = 1
feature_google = data_google.columns[4]
target_google = data_google.columns[0]
new_data_google = split_sequence(stock_data_g, ts_col=target_google , x_col=feature_google, n_steps=sequence_len, n_outs=n_outs)
new_data_google.head(10)

,Date,Close (t-30),Close (t-29),Close (t-28),Close (t-27),Close (t-26),Close (t-25),Close (t-24),Close (t-23),Close (t-22),...,Close (t-9),Close (t-8),Close (t-7),Close (t-6),Close (t-5),Close (t-4),Close (t-3),Close (t-2),Close (t-1),Close (t)
0,2010-02-17,0.090365,0.089057,0.081603,0.074892,0.078646,0.078214,0.073177,0.071570,0.072878,...,0.049642,0.042989,0.045126,0.046159,0.047567,0.046623,0.047548,0.045993,0.049870,0.048405
1,2010-02-18,0.089057,0.081603,0.074892,0.078646,0.078214,0.073177,0.071570,0.072878,0.068210,...,0.042989,0.045126,0.046159,0.047567,0.046623,0.047548,0.045993,0.049870,0.048405,0.050780
2,2010-02-19,0.081603,0.074892,0.078646,0.078214,0.073177,0.071570,0.072878,0.068210,0.071821,...,0.045126,0.046159,0.047567,0.046623,0.047548,0.045993,0.049870,0.048405,0.050780,0.049614
3,2010-02-22,0.074892,0.078646,0.078214,0.073177,0.071570,0.072878,0.068210,0.071821,0.068404,...,0.046159,0.047567,0.046623,0.047548,0.045993,0.049870,0.048405,0.050780,0.049614,0.050581
4,2010-02-23,0.078646,0.078214,0.073177,0.071570,0.072878,0.068210,0.071821,0.068404,0.069622,...,0.047567,0.046623,0.047548,0.045993,0.049870,0.048405,0.050780,0.049614,0.050581,0.046917
5,2010-02-24,0.078214,0.073177,0.071570,0.072878,0.068210,0.071821,0.068404,0.069622,0.053997,...,0.046623,0.047548,0.045993,0.049870,0.048405,0.050780,0.049614,0.050581,0.046917,0.045211
6,2010-02-25,0.073177,0.071570,0.072878,0.068210,0.071821,0.068404,0.069622,0.053997,0.049254,...,0.047548,0.045993,0.049870,0.048405,0.050780,0.049614,0.050581,0.046917,0.045211,0.042823
7,2010-02-26,0.071570,0.072878,0.068210,0.071821,0.068404,0.069622,0.053997,0.049254,0.050400,...,0.045993,0.049870,0.048405,0.050780,0.049614,0.050581,0.046917,0.045211,0.042823,0.042998
8,2010-03-01,0.072878,0.068210,0.071821,0.068404,0.069622,0.053997,0.049254,0.050400,0.050249,...,0.049870,0.048405,0.050780,0.049614,0.050581,0.046917,0.045211,0.042823,0.042998,0.045789
9,2010-03-02,0.068210,0.071821,0.068404,0.069622,0.053997,0.049254,0.050400,0.050249,0.046548,...,0.048405,0.050780,0.049614,0.050581,0.046917,0.045211,0.042823,0.042998,0.045789,0.049756


In [ ]:
print('******** Google Dataset Info **********')

x_cols_google= new_data_google.columns[1:-n_outs]
y_cols_google = new_data_google.columns[-n_outs:]
print('Features: \n%s' % str(list(x_cols_google)))
print('Target: \n%s' % str(list(y_cols_google)))


print('******** Apple Dataset Info **********')

x_cols_apple= new_data_apple.columns[1:-n_outs]
y_cols_apple = new_data_apple.columns[-n_outs:]
print('Features: \n%s' % str(list(x_cols_apple)))
print('Target: \n%s' % str(list(y_cols_apple)))

******** Google Dataset Info **********
Features: 
['Close (t-30)', 'Close (t-29)', 'Close (t-28)', 'Close (t-27)', 'Close (t-26)', 'Close (t-25)', 'Close (t-24)', 'Close (t-23)', 'Close (t-22)', 'Close (t-21)', 'Close (t-20)', 'Close (t-19)', 'Close (t-18)', 'Close (t-17)', 'Close (t-16)', 'Close (t-15)', 'Close (t-14)', 'Close (t-13)', 'Close (t-12)', 'Close (t-11)', 'Close (t-10)', 'Close (t-9)', 'Close (t-8)', 'Close (t-7)', 'Close (t-6)', 'Close (t-5)', 'Close (t-4)', 'Close (t-3)', 'Close (t-2)', 'Close (t-1)']
Target: 
['Close (t)']
******** Apple Dataset Info **********
Features: 
['Close (t-30)', 'Close (t-29)', 'Close (t-28)', 'Close (t-27)', 'Close (t-26)', 'Close (t-25)', 'Close (t-24)', 'Close (t-23)', 'Close (t-22)', 'Close (t-21)', 'Close (t-20)', 'Close (t-19)', 'Close (t-18)', 'Close (t-17)', 'Close (t-16)', 'Close (t-15)', 'Close (t-14)', 'Close (t-13)', 'Close (t-12)', 'Close (t-11)', 'Close (t-10)', 'Close (t-9)', 'Close (t-8)', 'Close (t-7)', 'Close (t-6)', 'Close 

In [ ]:
new_data_google[new_data_google.columns[1:]] = new_data_google[new_data_google.columns[1:]].astype('float32')
new_data_apple[new_data_apple.columns[1:]] = new_data_apple[new_data_apple.columns[1:]].astype('float32')

In [ ]:
def timeseries_split(data, ts_col, valid_step=100, valid_range=5, test_size=0.2):
    _data = data.copy()
    _data[ts_col] = pd.to_datetime(_data[ts_col], utc=False)

    train_test_splitter = len(_data) - int(len(_data) * test_size)

    _train_data = _data.iloc[:train_test_splitter]
    test_data = _data.iloc[train_test_splitter:]

    train_rngs = [[i, i + valid_step] for i in list(range(valid_step, len(_train_data), valid_step))]

    valid_rngs = train_rngs[5:][::valid_range]
    valid_data = []
    for valid_rng in valid_rngs:
        train_rngs.pop(train_rngs.index(valid_rng))
        valid_data.append(_train_data.iloc[valid_rng[0]: valid_rng[1]])
    
    train_data = []
    for train_rng in train_rngs:
        train_data.append(_train_data.iloc[train_rng[0]: train_rng[1]])

    train_data = pd.concat(train_data, 0)
    valid_data = pd.concat(valid_data, 0)
    

    train_data = train_data.reset_index(drop=True)
    valid_data = valid_data.reset_index(drop=True)
    test_data = test_data.reset_index(drop=True)

    return train_data, valid_data, test_data
    

In [ ]:
new_data_apple

,Date,Close (t-30),Close (t-29),Close (t-28),Close (t-27),Close (t-26),Close (t-25),Close (t-24),Close (t-23),Close (t-22),...,Close (t-9),Close (t-8),Close (t-7),Close (t-6),Close (t-5),Close (t-4),Close (t-3),Close (t-2),Close (t-1),Close (t)
0,2010-02-17,0.015330,0.015589,0.013208,0.012936,0.013913,0.012608,0.010939,0.012985,0.012133,...,0.005012,0.000000,0.002381,0.001445,0.002890,0.002143,0.004621,0.005815,0.007924,0.007330
1,2010-02-18,0.015589,0.013208,0.012936,0.013913,0.012608,0.010939,0.012985,0.012133,0.009690,...,0.000000,0.002381,0.001445,0.002890,0.002143,0.004621,0.005815,0.007924,0.007330,0.007595
2,2010-02-19,0.013208,0.012936,0.013913,0.012608,0.010939,0.012985,0.012133,0.009690,0.016050,...,0.002381,0.001445,0.002890,0.002143,0.004621,0.005815,0.007924,0.007330,0.007595,0.006716
3,2010-02-22,0.012936,0.013913,0.012608,0.010939,0.012985,0.012133,0.009690,0.016050,0.013739,...,0.001445,0.002890,0.002143,0.004621,0.005815,0.007924,0.007330,0.007595,0.006716,0.005843
4,2010-02-23,0.013913,0.012608,0.010939,0.012985,0.012133,0.009690,0.016050,0.013739,0.011184,...,0.002890,0.002143,0.004621,0.005815,0.007924,0.007330,0.007595,0.006716,0.005843,0.003498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,2018-12-24,0.884770,0.865125,0.814792,0.805311,0.778776,0.801304,0.811664,0.774182,0.730788,...,0.689984,0.692280,0.701321,0.674590,0.667065,0.677473,0.652160,0.632320,0.602510,0.583452
2230,2018-12-26,0.865125,0.814792,0.805311,0.778776,0.801304,0.811664,0.774182,0.730788,0.729811,...,0.692280,0.701321,0.674590,0.667065,0.677473,0.652160,0.632320,0.602510,0.583452,0.633981
2231,2018-12-27,0.814792,0.805311,0.778776,0.801304,0.811664,0.774182,0.730788,0.729811,0.707869,...,0.701321,0.674590,0.667065,0.677473,0.652160,0.632320,0.602510,0.583452,0.633981,0.628997
2232,2018-12-28,0.805311,0.778776,0.801304,0.811664,0.774182,0.730788,0.729811,0.707869,0.719255,...,0.674590,0.667065,0.677473,0.652160,0.632320,0.602510,0.583452,0.633981,0.628997,0.629388


In [ ]:
a = timeseries_split(new_data_apple, target_apple, valid_step=35, valid_range=3, test_size=0.25)
print(a)

(           Date  Close (t-30)  Close (t-29)  Close (t-28)  Close (t-27)  \
0    2010-04-08      0.006011      0.006946      0.008775      0.011826   
1    2010-04-09      0.006946      0.008775      0.011826      0.011728   
2    2010-04-12      0.008775      0.011826      0.011728      0.012063   
3    2010-04-13      0.011826      0.011728      0.012063      0.013027   
4    2010-04-14      0.011728      0.012063      0.013027      0.018779   
...         ...           ...           ...           ...           ...   
1146 2016-10-05      0.397853      0.393845      0.391598      0.388519   
1147 2016-10-06      0.393845      0.391598      0.388519      0.387932   
1148 2016-10-07      0.391598      0.388519      0.387932      0.383925   
1149 2016-10-10      0.388519      0.387932      0.383925      0.384414   
1150 2016-10-11      0.387932      0.383925      0.384414      0.387493   

      Close (t-26)  Close (t-25)  Close (t-24)  Close (t-23)  Close (t-22)  \
0         0.011728  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only



In [ ]:
# Making Train and Test Set for Apple Stock

train_data_apple, valid_data_apple, test_data_apple = timeseries_split(new_data_apple, target_apple, valid_step=35, valid_range=3, test_size=0.25)

x_train_apple, y_train_apple = train_data_apple.loc[:, x_cols_apple].values, train_data_apple.loc[:, y_cols_apple].values
x_valid_apple, y_valid_apple = valid_data_apple.loc[:, x_cols_apple].values, valid_data_apple.loc[:, y_cols_apple].values
x_test_apple, y_test_apple = test_data_apple.loc[:, x_cols_apple].values, test_data_apple.loc[:, y_cols_apple].values

x_train_apple = np.expand_dims(x_train_apple, -1)
x_valid_apple = np.expand_dims(x_valid_apple, -1)
x_test_apple = np.expand_dims(x_test_apple, -1)

x_train_apple = x_train_apple.astype(np.float32)
x_valid_apple = x_valid_apple.astype(np.float32)
x_test_apple = x_test_apple.astype(np.float32)

y_train_apple = y_train_apple.astype(np.float32)
y_valid_apple = y_valid_apple.astype(np.float32)
y_test_apple = y_test_apple.astype(np.float32)

print('Training %s %s' % (str(x_train_apple.shape), str(y_train_apple.shape)))
print('Validating %s %s' % (str(x_valid_apple.shape), str(y_valid_apple.shape)))
print('Testing %s %s' % (str(x_test_apple.shape), str(y_test_apple.shape)))

fig = make_subplots()

fig.add_trace(
    go.Scatter(x=train_data_apple['Date'], y=train_data_apple['Close (t)'], name='Close (t) Train'),
)

fig.add_trace(
    go.Scatter(x=valid_data_apple['Date'], y=valid_data_apple['Close (t)'], name='Close (t) Valid'),
)

fig.add_trace(
    go.Scatter(x=test_data_apple['Date'], y=test_data_apple['Close (t)'], name='Close (t) Test'),
)

fig.update_layout(
    title_text='Train/Valid/Test Split of Apple'
)

fig.show()

Training (1151, 30, 1) (1151, 1)
Validating (490, 30, 1) (490, 1)
Testing (558, 30, 1) (558, 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only



In [ ]:
# Making Train and Test Set for Google Stock

train_data_google, valid_data_google, test_data_google = timeseries_split(new_data_google, target_google, valid_step=40, valid_range=3, test_size=0.25)

x_train_google, y_train_google = train_data_google.loc[:, x_cols_google].values, train_data_google.loc[:, y_cols_google].values
x_valid_google, y_valid_google = valid_data_google.loc[:, x_cols_google].values, valid_data_google.loc[:, y_cols_google].values
x_test_google, y_test_google = test_data_google.loc[:, x_cols_google].values, test_data_google.loc[:, y_cols_google].values

x_train_google = np.expand_dims(x_train_google, -1)
x_valid_google = np.expand_dims(x_valid_google, -1)
x_test_google = np.expand_dims(x_test_google, -1)

x_train_google = x_train_google.astype(np.float32)
x_valid_google = x_valid_google.astype(np.float32)
x_test_google = x_test_google.astype(np.float32)

y_train_google = y_train_google.astype(np.float32)
y_valid_google = y_valid_google.astype(np.float32)
y_test_google = y_test_google.astype(np.float32)

print('Training %s %s' % (str(x_train_google.shape), str(y_train_google.shape)))
print('Validating %s %s' % (str(x_valid_google.shape), str(y_valid_google.shape)))
print('Testing %s %s' % (str(x_test_google.shape), str(y_test_google.shape)))
fig = make_subplots()

fig.add_trace(
    go.Scatter(x=train_data_apple['Date'], y=train_data_apple['Close (t)'], name='Close (t) Train'),
)

fig.add_trace(
    go.Scatter(x=valid_data_apple['Date'], y=valid_data_apple['Close (t)'], name='Close (t) Valid'),
)

fig.add_trace(
    go.Scatter(x=test_data_apple['Date'], y=test_data_apple['Close (t)'], name='Close (t) Test'),
)

fig.update_layout(
    title_text='Train/Valid/Test Split of Apple'
)

fig.show()

Training (1156, 30, 1) (1156, 1)
Validating (480, 30, 1) (480, 1)
Testing (558, 30, 1) (558, 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: FutureWarning:

In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only



In [ ]:
def build_model(n_timesteps, n_features, n_outputs=1, rnn_units=None, dropout_rate=0.0, lr=0.001, optimizer_func='adam', loss_func='mse'):
    rnn_units = rnn_units if isinstance(rnn_units, list) else [100, 100]

    inputs = tf.keras.layers.Input(shape=[n_timesteps, n_features], name='inputs')

    x = inputs

    for units in rnn_units[:-1]:
        x = tf.keras.layers.LSTM(units, return_sequences=True)(x)
        x = tf.keras.layers.Dropout(rate=dropout_rate)(x)
    
    x = tf.keras.layers.LSTM(rnn_units[-1], return_sequences=False)(x)
    x = tf.keras.layers.Dropout(rate=dropout_rate)(x)

    outputs = tf.keras.layers.Dense(n_outputs, activation='linear')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    if optimizer_func == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(lr=lr,momentum=0.9)
    elif optimizer_func == 'adagrad':
        opt = tf.keras.optimizers.Adagrad(lr=lr)
    elif optimizer_func == 'adam':
        opt = tf.keras.optimizers.Adam(lr=lr)

    model.compile(optimizer=opt, loss=loss_func)

    # opt = tf.keras.optimizers.Adam(lr=lr)
    # model.compile(optimizer=opt, loss='mse')

    return model

In [ ]:
def build_model_gru(n_timesteps, n_features, n_outputs=1, rnn_units=None, dropout_rate=0.0, lr=0.001, optimizer_func='rmsprop', loss_func='mse'):
    rnn_units = rnn_units if isinstance(rnn_units, list) else [100, 100]

    inputs = tf.keras.layers.Input(shape=[n_timesteps, n_features], name='inputs')

    x = inputs

    for units in rnn_units[:-1]:
        x = tf.keras.layers.GRU(units, return_sequences=True)(x)
        x = tf.keras.layers.Dropout(rate=dropout_rate)(x)
    
    x = tf.keras.layers.GRU(rnn_units[-1], return_sequences=False)(x)
    x = tf.keras.layers.Dropout(rate=dropout_rate)(x)

    outputs = tf.keras.layers.Dense(n_outputs,activation = 'linear')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    if optimizer_func == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(lr=lr,momentum=0.9)
    elif optimizer_func == 'adagrad':
        opt = tf.keras.optimizers.Adagrad(lr=lr)
    elif optimizer_func == 'adam':
        opt = tf.keras.optimizers.Adam(lr=lr)

    model.compile(optimizer=opt, loss=loss_func)

    return model

In [ ]:
def build_model_simplernn(n_timesteps, n_features, n_outputs=1, rnn_units=None, dropout_rate=0.0, lr=0.001, optimizer_func='adam', loss_func='mse'):
    rnn_units = rnn_units if isinstance(rnn_units, list) else [100, 100]

    inputs = tf.keras.layers.Input(shape=[n_timesteps, n_features], name='inputs')

    x = inputs

    for units in rnn_units[:-1]:
        x = tf.keras.layers.SimpleRNN(units, return_sequences=True)(x)
        x = tf.keras.layers.Dropout(rate=dropout_rate)(x)
    
    x = tf.keras.layers.SimpleRNN(rnn_units[-1], return_sequences=False)(x)
    x = tf.keras.layers.Dropout(rate=dropout_rate)(x)

    outputs = tf.keras.layers.Dense(n_outputs,activation = 'linear')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    if optimizer_func == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(lr=lr,momentum=0.9)
    elif optimizer_func == 'adagrad':
        opt = tf.keras.optimizers.Adagrad(lr=lr)
    elif optimizer_func == 'adam':
        opt = tf.keras.optimizers.Adam(lr=lr)

    model.compile(optimizer=opt, loss=loss_func)


    # opt = tf.keras.optimizers.Adam(lr=lr)
    # model.compile(optimizer=opt, loss='mse')

    return model

In [ ]:
print('** APPLE **')
[n_timesteps_apple, n_features_apple], n_outputs_apple = x_train_apple.shape[1:], y_train_apple.shape[1]
print('#%d Timesteps, #%d Features, #%d Target' % (n_timesteps_apple, n_features_apple, n_outputs_apple))

print('** GOOGLE **')
[n_timesteps_google, n_features_google], n_outputs_google = x_train_google.shape[1:], y_train_google.shape[1]
print('#%d Timesteps, #%d Features, #%d Target' % (n_timesteps_google, n_features_google, n_outputs_google))

** APPLE **
#30 Timesteps, #1 Features, #1 Target
** GOOGLE **
#30 Timesteps, #1 Features, #1 Target


In [ ]:
def prediction_point_by_point(model, x):
    predicted = model.predict(x)
    return predicted


def error(y_true,y_pred):
    check_array(y_true, y_pred)
    mse_error = mse(y_true,y_pred)
    rmse_error = sqrt(mse_error)
    mape_error = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return 'MSE_ERROR: %.2f\nRMSE_ERROR: %.2f\nMAPE_ERROR: %.2f' % (mse_error,rmse_error,mape_error)

def pred(model_name, optimizer_func, loss_func, dropout_rate):

    print('*** APPLE ***')
    
    if model_name == 'gru':
        model_1 = build_model_gru(n_timesteps_apple, n_features_apple, n_outputs=n_outputs_apple,
                            rnn_units=[200, 200, 100], dropout_rate=dropout_rate, lr=0.001, optimizer_func=optimizer_func, loss_func=loss_func)
    if model_name == 'lstm':
        model_1 = build_model(n_timesteps_apple, n_features_apple, n_outputs=n_outputs_apple,
                            rnn_units=[200, 200, 100], dropout_rate=dropout_rate, lr=0.001, optimizer_func=optimizer_func, loss_func=loss_func)        
    if model_name == 'rnn':
        model_1 = build_model_simplernn(n_timesteps_apple, n_features_apple, n_outputs=n_outputs_apple,
                            rnn_units=[200, 200, 100], dropout_rate=dropout_rate, lr=0.001, optimizer_func=optimizer_func, loss_func=loss_func)    
    
    model_1.summary()

    start_time = datetime.now()

    apple_fit = model_1.fit(
        x_train_apple, y_train_apple, 
        validation_data=(x_valid_apple, y_valid_apple),
        batch_size=128,
        verbose=1,
        epochs=80)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

    history_dict = apple_fit.history
    # history_dict.keys()

    loss = history_dict['loss']
    val_loss = history_dict['val_loss']
    epochs = list(range(1, len(loss) + 1))



    fig = make_subplots()
    fig.add_trace(
        go.Scatter(x=epochs, y=loss, name='Training Loss'),
    )
    fig.add_trace(
        go.Scatter(x=epochs, y=val_loss, name='Validation loss'),
    )
    fig.update_layout(
        title_text='Training/Validation Loss for Apple Stock'
    )
    fig.show()


    print('*** GOOGLE ***')

    if model_name == 'gru':
        model_2 = build_model_gru(n_timesteps_google, n_features_google, n_outputs=n_outputs_google,
                            rnn_units=[200, 200, 100], dropout_rate=dropout_rate, lr=0.001, optimizer_func=optimizer_func, loss_func=loss_func)
    if model_name == 'lstm':
        model_2 = build_model(n_timesteps_google, n_features_google, n_outputs=n_outputs_google,
                            rnn_units=[200, 200, 100], dropout_rate=dropout_rate, lr=0.001, optimizer_func=optimizer_func, loss_func=loss_func)        
    if model_name == 'rnn':
        model_2 = build_model_simplernn(n_timesteps_google, n_features_google, n_outputs=n_outputs_google,
                            rnn_units=[200, 200, 100], dropout_rate=dropout_rate, lr=0.001, optimizer_func=optimizer_func, loss_func=loss_func)    

    model_2.summary()

    start_time = datetime.now()

    google_fit = model_2.fit(
        x_train_google, y_train_google, 
        validation_data=(x_valid_google, y_valid_google),
        batch_size=128,
        verbose=1,
        epochs=80)

    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))

    history_dict = google_fit.history
    # history_dict.keys()

    loss = history_dict['loss']
    val_loss = history_dict['val_loss']
    epochs = list(range(1, len(loss) + 1))



    fig = make_subplots()
    fig.add_trace(
        go.Scatter(x=epochs, y=loss, name='Training Loss'),
    )
    fig.add_trace(
        go.Scatter(x=epochs, y=val_loss, name='Validation loss'),
    )
    fig.update_layout(
        title_text='Training/Validation Loss for Google Stock'
    )
    fig.show()


    #Accuracy

    loss_apple= model_1.evaluate(x_train_apple, y_train_apple, verbose=1)
    accuracy = 1 - loss_apple
    print("accuracy of apple prediction is:", accuracy)

    loss_google= model_2.evaluate(x_train_google, y_train_google, verbose=1)
    accuracy = 1 - loss_google
    print("accuracy of google prediction is:" ,accuracy)



    #Accuracy

    loss_apple= model_1.evaluate(x_valid_apple, y_valid_apple, verbose=1)
    accuracy = 1 - loss_apple
    print("Validation accuracy of apple prediction is:", accuracy)

    loss_google= model_2.evaluate(x_valid_google, y_valid_google, verbose=1)
    accuracy = 1 - loss_google
    print("Validation accuracy of google prediction is:" ,accuracy)



    xx_google = x_test_google
    yy_google = y_test_google
    tt_google = test_data_google

    if len(yy_google.shape) > 1:
        yy_google = scaler.inverse_transform(yy_google)
    else:
        yy_google = scaler.inverse_transform(yy_google.reshape(-1, 1)).flatten()
        
    pp_google = prediction_point_by_point(model_2, xx_google)

    if len(pp_google.shape) > 1:
        pp_google = scaler.inverse_transform(pp_google)
    else:
        pp_google = scaler.inverse_transform(pp_google.reshape(-1, 1)).flatten()


    xx_apple = x_test_apple
    yy_apple = y_test_apple
    tt_apple = test_data_apple

    if len(yy_apple.shape) > 1:
        yy_apple = scaler.inverse_transform(yy_apple)
    else:
        yy_apple = scaler.inverse_transform(yy_apple.reshape(-1, 1)).flatten()
        
    pp_apple = prediction_point_by_point(model_1, xx_apple)

    if len(pp_apple.shape) > 1:
        pp_apple = scaler.inverse_transform(pp_apple)
    else:
        pp_apple = scaler.inverse_transform(pp_apple.reshape(-1, 1)).flatten()



    print('**** APPLE ERROR SUMMARY ***')
    apple_error = error(pp_apple,yy_apple)
    print(apple_error)
    print('**** GOOGLE ERROR SUMMARY ***')
    google_error = error(pp_google,yy_google)
    print(google_error)   


    #compare the model performance in predicting the price

    fig = make_subplots()

    fig.add_trace(
        go.Scatter(x=tt_apple['Date'], y=yy_apple[:, 0], name='True Target Apple'),
    )
    fig.add_trace(
        go.Scatter(x=tt_apple['Date'], y=pp_apple[:, 0], name='Predicted Apple'),
    )
    fig.add_trace(
        go.Scatter(x=tt_google['Date'], y=yy_google[:, 0], name='True Target Google'),
    )
    fig.add_trace(
        go.Scatter(x=tt_google['Date'], y=pp_google[:, 0], name='Predicted Google'),
    )

    fig.update_layout(
        title_text='Point-By-Point Prediction'
    )

    fig.show()     

In [ ]:
pred('gru','rmsprop','mse',0.0)

*** APPLE ***
Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_12 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_38 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_13 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_39 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_14 (GRU)                (None, 100)               90600     
                                                                 
 dropout_40 (Dropout)        (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 6s 135ms/step - loss: 0.0185 - val_loss: 0.0032
Epoch 2/80
9/9 [==============================] - 0s 25ms/step - loss: 0.0032 - val_loss: 0.0026
Epoch 3/80
9/9 [==============================] - 0s 24ms/step - loss: 0.0011 - val_loss: 9.3208e-04
Epoch 4/80
9/9 [==============================] - 0s 25ms/step - loss: 3.5346e-04 - val_loss: 2.3868e-04
Epoch 5/80
9/9 [==============================] - 0s 24ms/step - loss: 1.7558e-04 - val_loss: 2.3250e-04
Epoch 6/80
9/9 [==============================] - 0s 22ms/step - loss: 1.2418e-04 - val_loss: 1.4215e-04
Epoch 7/80
9/9 [==============================] - 0s 22ms/step - loss: 1.3640e-04 - val_loss: 1.4087e-04
Epoch 8/80
9/9 [==============================] - 0s 23ms/step - loss: 1.3112e-04 - val_loss: 1.0869e-04
Epoch 9/80
9/9 [==============================] - 0s 22ms/step - loss: 3.5438e-04 - val_loss: 1.8841e-04
Epoch 10/80
9/9 [==============================] - 0s 25ms/step - loss: 2.7877e-04 - 

*** GOOGLE ***
Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_15 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_41 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_16 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_42 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_17 (GRU)                (None, 100)               90600     
                                                                 
 dropout_43 (Dropout)        (None, 100)   

36/36 [==============================] - 0s 5ms/step - loss: 7.2952e-05
accuracy of apple prediction is: 0.9999270481057465
37/37 [==============================] - 0s 8ms/step - loss: 1.7761e-04
accuracy of google prediction is: 0.9998223939473974
16/16 [==============================] - 0s 7ms/step - loss: 8.7466e-05
Validation accuracy of apple prediction is: 0.9999125335161807
15/15 [==============================] - 2s 7ms/step - loss: 1.2546e-04
Validation accuracy of google prediction is: 0.999874536471907
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 17.85
RMSE_ERROR: 4.22
MAPE_ERROR: 2.04
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 9.81
RMSE_ERROR: 3.13
MAPE_ERROR: 1.23


In [ ]:
pred('lstm','adam','mse',0.0)

*** APPLE ***
Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_18 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_44 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_19 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_45 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_20 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_46 (Dropout)        (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 6s 147ms/step - loss: 0.0171 - val_loss: 0.0011
Epoch 2/80
9/9 [==============================] - 0s 22ms/step - loss: 0.0029 - val_loss: 0.0023
Epoch 3/80
9/9 [==============================] - 0s 19ms/step - loss: 0.0011 - val_loss: 8.6877e-04
Epoch 4/80
9/9 [==============================] - 0s 17ms/step - loss: 7.8388e-04 - val_loss: 5.9282e-04
Epoch 5/80
9/9 [==============================] - 0s 18ms/step - loss: 3.8476e-04 - val_loss: 6.5643e-04
Epoch 6/80
9/9 [==============================] - 0s 17ms/step - loss: 3.3115e-04 - val_loss: 4.3711e-04
Epoch 7/80
9/9 [==============================] - 0s 17ms/step - loss: 2.9193e-04 - val_loss: 3.6352e-04
Epoch 8/80
9/9 [==============================] - 0s 18ms/step - loss: 2.6493e-04 - val_loss: 3.8022e-04
Epoch 9/80
9/9 [==============================] - 0s 18ms/step - loss: 2.6231e-04 - val_loss: 3.5259e-04
Epoch 10/80
9/9 [==============================] - 0s 17ms/step - loss: 2.5945e-04 - 

*** GOOGLE ***
Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_21 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_47 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_22 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_48 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_23 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_49 (Dropout)        (None, 100)   

36/36 [==============================] - 0s 5ms/step - loss: 2.0929e-04
accuracy of apple prediction is: 0.9997907065262552
37/37 [==============================] - 0s 5ms/step - loss: 2.3122e-04
accuracy of google prediction is: 0.9997687764407601
16/16 [==============================] - 0s 5ms/step - loss: 2.1767e-04
Validation accuracy of apple prediction is: 0.9997823279118165
15/15 [==============================] - 1s 5ms/step - loss: 1.6365e-04
Validation accuracy of google prediction is: 0.9998363541235449
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 49.56
RMSE_ERROR: 7.04
MAPE_ERROR: 3.63
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 28.19
RMSE_ERROR: 5.31
MAPE_ERROR: 2.19


In [ ]:
pred('rnn','adam','mse',0.0)

*** APPLE ***
Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_8 (SimpleRNN)    (None, 30, 200)           40400     
                                                                 
 dropout_50 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_9 (SimpleRNN)    (None, 30, 200)           80200     
                                                                 
 dropout_51 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_52 (Dropout)        (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 3s 137ms/step - loss: 1.0072 - val_loss: 0.0328
Epoch 2/80
9/9 [==============================] - 1s 98ms/step - loss: 0.0303 - val_loss: 0.0349
Epoch 3/80
9/9 [==============================] - 1s 99ms/step - loss: 0.0218 - val_loss: 0.0104
Epoch 4/80
9/9 [==============================] - 1s 92ms/step - loss: 0.0165 - val_loss: 0.0138
Epoch 5/80
9/9 [==============================] - 1s 99ms/step - loss: 0.0147 - val_loss: 0.0096
Epoch 6/80
9/9 [==============================] - 1s 97ms/step - loss: 0.0071 - val_loss: 0.0064
Epoch 7/80
9/9 [==============================] - 1s 94ms/step - loss: 0.0077 - val_loss: 0.0032
Epoch 8/80
9/9 [==============================] - 1s 101ms/step - loss: 0.0071 - val_loss: 0.0081
Epoch 9/80
9/9 [==============================] - 1s 100ms/step - loss: 0.0042 - val_loss: 0.0046
Epoch 10/80
9/9 [==============================] - 1s 91ms/step - loss: 0.0023 - val_loss: 0.0018
Epoch 11/80
9/9 [====================

*** GOOGLE ***
Model: "model_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_11 (SimpleRNN)   (None, 30, 200)           40400     
                                                                 
 dropout_53 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_12 (SimpleRNN)   (None, 30, 200)           80200     
                                                                 
 dropout_54 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_13 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_55 (Dropout)        (None, 100)   

36/36 [==============================] - 0s 9ms/step - loss: 7.8290e-04
accuracy of apple prediction is: 0.99921709834598
37/37 [==============================] - 0s 10ms/step - loss: 2.0854e-04
accuracy of google prediction is: 0.9997914568230044
16/16 [==============================] - 0s 9ms/step - loss: 7.6245e-04
Validation accuracy of apple prediction is: 0.9992375470465049
15/15 [==============================] - 0s 10ms/step - loss: 1.4804e-04
Validation accuracy of google prediction is: 0.9998519613291137
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 551.87
RMSE_ERROR: 23.49
MAPE_ERROR: 13.07
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 355.68
RMSE_ERROR: 18.86
MAPE_ERROR: 9.37


MSE vs. MAPE

In [ ]:
pred('gru','rmsprop','mape',0.0)

*** APPLE ***
Model: "model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_18 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_56 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_19 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_57 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_20 (GRU)                (None, 100)               90600     
                                                                 
 dropout_58 (Dropout)        (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 5s 136ms/step - loss: 90.3674 - val_loss: 34.8585
Epoch 2/80
9/9 [==============================] - 0s 25ms/step - loss: 50.1294 - val_loss: 55.4440
Epoch 3/80
9/9 [==============================] - 0s 19ms/step - loss: 49.1635 - val_loss: 28.7271
Epoch 4/80
9/9 [==============================] - 0s 19ms/step - loss: 27.0881 - val_loss: 21.6179
Epoch 5/80
9/9 [==============================] - 0s 16ms/step - loss: 22.0252 - val_loss: 14.5116
Epoch 6/80
9/9 [==============================] - 0s 16ms/step - loss: 16.5892 - val_loss: 12.7194
Epoch 7/80
9/9 [==============================] - 0s 16ms/step - loss: 13.4274 - val_loss: 9.9460
Epoch 8/80
9/9 [==============================] - 0s 16ms/step - loss: 8.8485 - val_loss: 9.6894
Epoch 9/80
9/9 [==============================] - 0s 16ms/step - loss: 8.9720 - val_loss: 8.2910
Epoch 10/80
9/9 [==============================] - 0s 16ms/step - loss: 9.9311 - val_loss: 6.5642
Epoch 11/80
9/9 [=========

*** GOOGLE ***
Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_21 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_59 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_22 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_60 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_23 (GRU)                (None, 100)               90600     
                                                                 
 dropout_61 (Dropout)        (None, 100)   

36/36 [==============================] - 0s 5ms/step - loss: 5.3441
accuracy of apple prediction is: -4.344135761260986
37/37 [==============================] - 0s 5ms/step - loss: 435831.2500
accuracy of google prediction is: -435830.25
16/16 [==============================] - 0s 5ms/step - loss: 4.0856
Validation accuracy of apple prediction is: -3.08559513092041
15/15 [==============================] - 1s 6ms/step - loss: 456.7757
Validation accuracy of google prediction is: -455.7756652832031
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 471.24
RMSE_ERROR: 21.71
MAPE_ERROR: 9.89
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 65217.86
RMSE_ERROR: 255.38
MAPE_ERROR: 336.69


In [ ]:
pred('lstm','adam','mape',0.0)

*** APPLE ***
Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_24 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_62 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_25 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_63 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_26 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_64 (Dropout)        (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 5s 150ms/step - loss: 41.0776 - val_loss: 20.0050
Epoch 2/80
9/9 [==============================] - 0s 26ms/step - loss: 13.7828 - val_loss: 9.9598
Epoch 3/80
9/9 [==============================] - 0s 21ms/step - loss: 9.9698 - val_loss: 6.2879
Epoch 4/80
9/9 [==============================] - 0s 19ms/step - loss: 8.0217 - val_loss: 6.2995
Epoch 5/80
9/9 [==============================] - 0s 19ms/step - loss: 6.7753 - val_loss: 6.0904
Epoch 6/80
9/9 [==============================] - 0s 19ms/step - loss: 7.2584 - val_loss: 7.0270
Epoch 7/80
9/9 [==============================] - 0s 18ms/step - loss: 6.7331 - val_loss: 5.5104
Epoch 8/80
9/9 [==============================] - 0s 19ms/step - loss: 6.0698 - val_loss: 5.3941
Epoch 9/80
9/9 [==============================] - 0s 19ms/step - loss: 6.0491 - val_loss: 5.3448
Epoch 10/80
9/9 [==============================] - 0s 17ms/step - loss: 6.2530 - val_loss: 5.3906
Epoch 11/80
9/9 [===================

*** GOOGLE ***
Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_27 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_65 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_28 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_66 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_29 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_67 (Dropout)        (None, 100)   

36/36 [==============================] - 0s 6ms/step - loss: 2.6879
accuracy of apple prediction is: -1.6878776550292969
37/37 [==============================] - 0s 6ms/step - loss: 753.1578
accuracy of google prediction is: -752.1578369140625
16/16 [==============================] - 0s 6ms/step - loss: 2.3464
Validation accuracy of apple prediction is: -1.346435546875
15/15 [==============================] - 1s 6ms/step - loss: 100.5767
Validation accuracy of google prediction is: -99.57673645019531
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 10.68
RMSE_ERROR: 3.27
MAPE_ERROR: 1.31
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 23597.74
RMSE_ERROR: 153.62
MAPE_ERROR: 556.16


In [ ]:
pred('rnn','adam','mape',0.0)

*** APPLE ***
Model: "model_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_14 (SimpleRNN)   (None, 30, 200)           40400     
                                                                 
 dropout_68 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_15 (SimpleRNN)   (None, 30, 200)           80200     
                                                                 
 dropout_69 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_16 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_70 (Dropout)        (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 4s 134ms/step - loss: 358.1678 - val_loss: 27.1661
Epoch 2/80
9/9 [==============================] - 1s 96ms/step - loss: 70.0196 - val_loss: 82.4241
Epoch 3/80
9/9 [==============================] - 1s 89ms/step - loss: 110.3845 - val_loss: 89.5379
Epoch 4/80
9/9 [==============================] - 1s 95ms/step - loss: 72.2132 - val_loss: 27.8309
Epoch 5/80
9/9 [==============================] - 1s 93ms/step - loss: 56.9633 - val_loss: 79.0609
Epoch 6/80
9/9 [==============================] - 1s 99ms/step - loss: 72.5412 - val_loss: 55.3321
Epoch 7/80
9/9 [==============================] - 1s 92ms/step - loss: 62.0265 - val_loss: 43.7627
Epoch 8/80
9/9 [==============================] - 1s 95ms/step - loss: 69.7268 - val_loss: 26.3818
Epoch 9/80
9/9 [==============================] - 1s 93ms/step - loss: 58.5870 - val_loss: 38.7014
Epoch 10/80
9/9 [==============================] - 1s 89ms/step - loss: 55.4352 - val_loss: 34.9982
Epoch 11/80
9/9 [

*** GOOGLE ***
Model: "model_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_17 (SimpleRNN)   (None, 30, 200)           40400     
                                                                 
 dropout_71 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_18 (SimpleRNN)   (None, 30, 200)           80200     
                                                                 
 dropout_72 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_19 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_73 (Dropout)        (None, 100)   

36/36 [==============================] - 0s 10ms/step - loss: 10.8519
accuracy of apple prediction is: -9.851941108703613
37/37 [==============================] - 0s 10ms/step - loss: 33845.4961
accuracy of google prediction is: -33844.49609375
16/16 [==============================] - 0s 10ms/step - loss: 11.6551
Validation accuracy of apple prediction is: -10.655147552490234
15/15 [==============================] - 0s 10ms/step - loss: 119.5743
Validation accuracy of google prediction is: -118.57428741455078
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 264.62
RMSE_ERROR: 16.27
MAPE_ERROR: 7.47
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 22435.80
RMSE_ERROR: 149.79
MAPE_ERROR: 474.14


RMSProp vs. Adam vs. ADAgrad

In [ ]:
pred('gru','rmsprop','mse',0.0)

*** APPLE ***
Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_24 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_74 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_25 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_75 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_26 (GRU)                (None, 100)               90600     
                                                                 
 dropout_76 (Dropout)        (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 5s 213ms/step - loss: 0.0256 - val_loss: 0.0145
Epoch 2/80
9/9 [==============================] - 0s 25ms/step - loss: 0.0056 - val_loss: 5.3348e-04
Epoch 3/80
9/9 [==============================] - 0s 19ms/step - loss: 0.0014 - val_loss: 4.1060e-04
Epoch 4/80
9/9 [==============================] - 0s 19ms/step - loss: 4.8818e-04 - val_loss: 1.0878e-04
Epoch 5/80
9/9 [==============================] - 0s 16ms/step - loss: 2.2355e-04 - val_loss: 2.9175e-04
Epoch 6/80
9/9 [==============================] - 0s 16ms/step - loss: 1.4860e-04 - val_loss: 1.0580e-04
Epoch 7/80
9/9 [==============================] - 0s 15ms/step - loss: 1.4587e-04 - val_loss: 1.1283e-04
Epoch 8/80
9/9 [==============================] - 0s 15ms/step - loss: 1.7146e-04 - val_loss: 1.0971e-04
Epoch 9/80
9/9 [==============================] - 0s 16ms/step - loss: 1.6727e-04 - val_loss: 1.3034e-04
Epoch 10/80
9/9 [==============================] - 0s 15ms/step - loss: 3.0655e-0

*** GOOGLE ***
Model: "model_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_27 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_77 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_28 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_78 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_29 (GRU)                (None, 100)               90600     
                                                                 
 dropout_79 (Dropout)        (None, 100)   

36/36 [==============================] - 0s 6ms/step - loss: 8.3679e-05
accuracy of apple prediction is: 0.9999163211614359
37/37 [==============================] - 0s 5ms/step - loss: 9.5243e-05
accuracy of google prediction is: 0.9999047574165161
16/16 [==============================] - 0s 5ms/step - loss: 9.7417e-05
Validation accuracy of apple prediction is: 0.9999025828074082
15/15 [==============================] - 1s 6ms/step - loss: 6.9748e-05
Validation accuracy of google prediction is: 0.9999302518335753
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 11.39
RMSE_ERROR: 3.38
MAPE_ERROR: 1.46
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 10.88
RMSE_ERROR: 3.30
MAPE_ERROR: 1.37


In [ ]:
pred('lstm','rmsprop','mse',0.0)

*** APPLE ***
Model: "model_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_30 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_80 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_31 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_81 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_32 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_82 (Dropout)        (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 5s 148ms/step - loss: 0.0261 - val_loss: 0.0038
Epoch 2/80
9/9 [==============================] - 0s 21ms/step - loss: 0.0047 - val_loss: 0.0029
Epoch 3/80
9/9 [==============================] - 0s 17ms/step - loss: 0.0015 - val_loss: 3.5959e-04
Epoch 4/80
9/9 [==============================] - 0s 18ms/step - loss: 4.9779e-04 - val_loss: 2.2441e-04
Epoch 5/80
9/9 [==============================] - 0s 17ms/step - loss: 2.4434e-04 - val_loss: 3.8232e-04
Epoch 6/80
9/9 [==============================] - 0s 22ms/step - loss: 2.0171e-04 - val_loss: 2.3611e-04
Epoch 7/80
9/9 [==============================] - 0s 17ms/step - loss: 1.7701e-04 - val_loss: 1.9563e-04
Epoch 8/80
9/9 [==============================] - 0s 18ms/step - loss: 1.8566e-04 - val_loss: 3.0310e-04
Epoch 9/80
9/9 [==============================] - 0s 17ms/step - loss: 2.1794e-04 - val_loss: 2.4246e-04
Epoch 10/80
9/9 [==============================] - 0s 16ms/step - loss: 2.0700e-04 - 

*** GOOGLE ***
Model: "model_28"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_33 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_83 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_34 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_84 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_35 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_85 (Dropout)        (None, 100)   

36/36 [==============================] - 0s 6ms/step - loss: 9.5286e-05
accuracy of apple prediction is: 0.9999047136661829
37/37 [==============================] - 0s 6ms/step - loss: 1.1866e-04
accuracy of google prediction is: 0.9998813411366427
16/16 [==============================] - 0s 5ms/step - loss: 1.2478e-04
Validation accuracy of apple prediction is: 0.9998752227402292
15/15 [==============================] - 1s 5ms/step - loss: 6.5756e-05
Validation accuracy of google prediction is: 0.9999342440569308
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 17.82
RMSE_ERROR: 4.22
MAPE_ERROR: 1.78
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 26.30
RMSE_ERROR: 5.13
MAPE_ERROR: 2.23


In [ ]:
pred('rnn','rmsprop','mse',0.0)

*** APPLE ***
Model: "model_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_20 (SimpleRNN)   (None, 30, 200)           40400     
                                                                 
 dropout_86 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_21 (SimpleRNN)   (None, 30, 200)           80200     
                                                                 
 dropout_87 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_22 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_88 (Dropout)        (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 3s 133ms/step - loss: 3.5055 - val_loss: 0.2480
Epoch 2/80
9/9 [==============================] - 1s 94ms/step - loss: 0.6570 - val_loss: 0.0134
Epoch 3/80
9/9 [==============================] - 1s 102ms/step - loss: 0.0108 - val_loss: 0.0551
Epoch 4/80
9/9 [==============================] - 1s 92ms/step - loss: 0.0249 - val_loss: 0.0069
Epoch 5/80
9/9 [==============================] - 1s 88ms/step - loss: 0.0317 - val_loss: 0.0894
Epoch 6/80
9/9 [==============================] - 1s 91ms/step - loss: 0.0346 - val_loss: 0.0258
Epoch 7/80
9/9 [==============================] - 1s 89ms/step - loss: 0.0472 - val_loss: 0.0323
Epoch 8/80
9/9 [==============================] - 1s 90ms/step - loss: 0.7257 - val_loss: 0.0580
Epoch 9/80
9/9 [==============================] - 1s 92ms/step - loss: 0.7311 - val_loss: 0.2573
Epoch 10/80
9/9 [==============================] - 1s 87ms/step - loss: 0.3617 - val_loss: 0.6827
Epoch 11/80
9/9 [=====================

*** GOOGLE ***
Model: "model_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_23 (SimpleRNN)   (None, 30, 200)           40400     
                                                                 
 dropout_89 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_24 (SimpleRNN)   (None, 30, 200)           80200     
                                                                 
 dropout_90 (Dropout)        (None, 30, 200)           0         
                                                                 
 simple_rnn_25 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_91 (Dropout)        (None, 100)   

36/36 [==============================] - 0s 9ms/step - loss: 0.0121
accuracy of apple prediction is: 0.9878912689164281
37/37 [==============================] - 0s 10ms/step - loss: 0.0269
accuracy of google prediction is: 0.9731134790927172
16/16 [==============================] - 0s 9ms/step - loss: 0.0129
Validation accuracy of apple prediction is: 0.987090116366744
15/15 [==============================] - 0s 10ms/step - loss: 0.0241
Validation accuracy of google prediction is: 0.9759480394423008
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 6447.96
RMSE_ERROR: 80.30
MAPE_ERROR: 82.97
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 13038.15
RMSE_ERROR: 114.18
MAPE_ERROR: 164.79


In [ ]:
pred('gru','adam','mse',0.0)

*** APPLE ***
Model: "model_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_30 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_92 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_31 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_93 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_32 (GRU)                (None, 100)               90600     
                                                                 
 dropout_94 (Dropout)        (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 5s 138ms/step - loss: 0.0195 - val_loss: 0.0067
Epoch 2/80
9/9 [==============================] - 0s 25ms/step - loss: 0.0028 - val_loss: 0.0012
Epoch 3/80
9/9 [==============================] - 0s 24ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 4/80
9/9 [==============================] - 0s 17ms/step - loss: 5.1817e-04 - val_loss: 1.5214e-04
Epoch 5/80
9/9 [==============================] - 0s 17ms/step - loss: 2.3981e-04 - val_loss: 2.8279e-04
Epoch 6/80
9/9 [==============================] - 0s 16ms/step - loss: 1.9012e-04 - val_loss: 1.3803e-04
Epoch 7/80
9/9 [==============================] - 0s 16ms/step - loss: 1.3083e-04 - val_loss: 1.8637e-04
Epoch 8/80
9/9 [==============================] - 0s 16ms/step - loss: 1.2105e-04 - val_loss: 1.6606e-04
Epoch 9/80
9/9 [==============================] - 0s 16ms/step - loss: 1.0940e-04 - val_loss: 1.3321e-04
Epoch 10/80
9/9 [==============================] - 0s 16ms/step - loss: 1.0664e-04 - val_

*** GOOGLE ***
Model: "model_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_33 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_95 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_34 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_96 (Dropout)        (None, 30, 200)           0         
                                                                 
 gru_35 (GRU)                (None, 100)               90600     
                                                                 
 dropout_97 (Dropout)        (None, 100)   

36/36 [==============================] - 0s 5ms/step - loss: 5.8618e-05
accuracy of apple prediction is: 0.9999413824771182
37/37 [==============================] - 0s 5ms/step - loss: 6.8557e-05
accuracy of google prediction is: 0.9999314434244297
16/16 [==============================] - 0s 5ms/step - loss: 8.4527e-05
Validation accuracy of apple prediction is: 0.9999154730830924
15/15 [==============================] - 1s 5ms/step - loss: 4.0262e-05
Validation accuracy of google prediction is: 0.9999597384558001
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 9.11
RMSE_ERROR: 3.02
MAPE_ERROR: 1.21
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 10.26
RMSE_ERROR: 3.20
MAPE_ERROR: 1.30


In [ ]:
pred('lstm','adam','mse',0.0)

*** APPLE ***
Model: "model_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_36 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_98 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_37 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_99 (Dropout)        (None, 30, 200)           0         
                                                                 
 lstm_38 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_100 (Dropout)       (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 5s 146ms/step - loss: 0.0214 - val_loss: 0.0035
Epoch 2/80
9/9 [==============================] - 0s 22ms/step - loss: 0.0029 - val_loss: 0.0028
Epoch 3/80
9/9 [==============================] - 0s 20ms/step - loss: 0.0016 - val_loss: 4.9062e-04
Epoch 4/80
9/9 [==============================] - 0s 17ms/step - loss: 5.8434e-04 - val_loss: 0.0012
Epoch 5/80
9/9 [==============================] - 0s 17ms/step - loss: 4.7729e-04 - val_loss: 4.0717e-04
Epoch 6/80
9/9 [==============================] - 0s 18ms/step - loss: 3.3491e-04 - val_loss: 3.7802e-04
Epoch 7/80
9/9 [==============================] - 0s 17ms/step - loss: 3.0174e-04 - val_loss: 3.6498e-04
Epoch 8/80
9/9 [==============================] - 0s 18ms/step - loss: 2.7926e-04 - val_loss: 3.9265e-04
Epoch 9/80
9/9 [==============================] - 0s 16ms/step - loss: 2.7260e-04 - val_loss: 4.0556e-04
Epoch 10/80
9/9 [==============================] - 0s 17ms/step - loss: 2.7147e-04 - val_

*** GOOGLE ***
Model: "model_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_39 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_101 (Dropout)       (None, 30, 200)           0         
                                                                 
 lstm_40 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_102 (Dropout)       (None, 30, 200)           0         
                                                                 
 lstm_41 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_103 (Dropout)       (None, 100)   

36/36 [==============================] - 0s 5ms/step - loss: 1.5405e-04
accuracy of apple prediction is: 0.9998459490598179
37/37 [==============================] - 0s 5ms/step - loss: 1.8160e-04
accuracy of google prediction is: 0.9998184005235089
16/16 [==============================] - 0s 6ms/step - loss: 2.0724e-04
Validation accuracy of apple prediction is: 0.9997927623335272
15/15 [==============================] - 1s 6ms/step - loss: 1.0498e-04
Validation accuracy of google prediction is: 0.9998950216759113
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 27.51
RMSE_ERROR: 5.25
MAPE_ERROR: 2.18
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 27.78
RMSE_ERROR: 5.27
MAPE_ERROR: 2.24


In [ ]:
pred('rnn','adam','mse',0.0)

*** APPLE ***
Model: "model_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_26 (SimpleRNN)   (None, 30, 200)           40400     
                                                                 
 dropout_104 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_27 (SimpleRNN)   (None, 30, 200)           80200     
                                                                 
 dropout_105 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_28 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_106 (Dropout)       (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 4s 134ms/step - loss: 0.9138 - val_loss: 0.0545
Epoch 2/80
9/9 [==============================] - 1s 90ms/step - loss: 0.0373 - val_loss: 0.0173
Epoch 3/80
9/9 [==============================] - 1s 97ms/step - loss: 0.0119 - val_loss: 0.0133
Epoch 4/80
9/9 [==============================] - 1s 95ms/step - loss: 0.0052 - val_loss: 0.0065
Epoch 5/80
9/9 [==============================] - 1s 96ms/step - loss: 0.0027 - val_loss: 0.0021
Epoch 6/80
9/9 [==============================] - 1s 102ms/step - loss: 0.0011 - val_loss: 6.6405e-04
Epoch 7/80
9/9 [==============================] - 1s 95ms/step - loss: 4.9404e-04 - val_loss: 6.4938e-04
Epoch 8/80
9/9 [==============================] - 1s 107ms/step - loss: 4.2573e-04 - val_loss: 3.3997e-04
Epoch 9/80
9/9 [==============================] - 1s 86ms/step - loss: 3.2690e-04 - val_loss: 3.1592e-04
Epoch 10/80
9/9 [==============================] - 1s 99ms/step - loss: 2.9162e-04 - val_loss: 2.9138e-04
E

*** GOOGLE ***
Model: "model_36"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_29 (SimpleRNN)   (None, 30, 200)           40400     
                                                                 
 dropout_107 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_30 (SimpleRNN)   (None, 30, 200)           80200     
                                                                 
 dropout_108 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_31 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_109 (Dropout)       (None, 100)   

36/36 [==============================] - 0s 10ms/step - loss: 1.5730e-04
accuracy of apple prediction is: 0.9998427036916837
37/37 [==============================] - 0s 10ms/step - loss: 8.4231e-04
accuracy of google prediction is: 0.9991576891043223
16/16 [==============================] - 0s 9ms/step - loss: 2.2208e-04
Validation accuracy of apple prediction is: 0.9997779208351858
15/15 [==============================] - 0s 10ms/step - loss: 7.3571e-04
Validation accuracy of google prediction is: 0.9992642946308479
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 72.25
RMSE_ERROR: 8.50
MAPE_ERROR: 3.36
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 353.69
RMSE_ERROR: 18.81
MAPE_ERROR: 10.14


In [ ]:
pred('gru','adagrad','mse',0.0)

*** APPLE ***
Model: "model_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_39 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_113 (Dropout)       (None, 30, 200)           0         
                                                                 
 gru_40 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_114 (Dropout)       (None, 30, 200)           0         
                                                                 
 gru_41 (GRU)                (None, 100)               90600     
                                                                 
 dropout_115 (Dropout)       (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adagrad.py:74: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 5s 129ms/step - loss: 0.0542 - val_loss: 0.0413
Epoch 2/80
9/9 [==============================] - 0s 20ms/step - loss: 0.0320 - val_loss: 0.0261
Epoch 3/80
9/9 [==============================] - 0s 17ms/step - loss: 0.0225 - val_loss: 0.0189
Epoch 4/80
9/9 [==============================] - 0s 16ms/step - loss: 0.0181 - val_loss: 0.0152
Epoch 5/80
9/9 [==============================] - 0s 16ms/step - loss: 0.0160 - val_loss: 0.0133
Epoch 6/80
9/9 [==============================] - 0s 16ms/step - loss: 0.0150 - val_loss: 0.0122
Epoch 7/80
9/9 [==============================] - 0s 15ms/step - loss: 0.0144 - val_loss: 0.0115
Epoch 8/80
9/9 [==============================] - 0s 15ms/step - loss: 0.0141 - val_loss: 0.0111
Epoch 9/80
9/9 [==============================] - 0s 16ms/step - loss: 0.0138 - val_loss: 0.0108
Epoch 10/80
9/9 [==============================] - 0s 16ms/step - loss: 0.0137 - val_loss: 0.0106
Epoch 11/80
9/9 [======================

*** GOOGLE ***
Model: "model_39"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_42 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_116 (Dropout)       (None, 30, 200)           0         
                                                                 
 gru_43 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_117 (Dropout)       (None, 30, 200)           0         
                                                                 
 gru_44 (GRU)                (None, 100)               90600     
                                                                 
 dropout_118 (Dropout)       (None, 100)   

36/36 [==============================] - 0s 6ms/step - loss: 0.0067
accuracy of apple prediction is: 0.9932706207036972
37/37 [==============================] - 0s 6ms/step - loss: 0.0092
accuracy of google prediction is: 0.9908397132530808
16/16 [==============================] - 0s 6ms/step - loss: 0.0051
Validation accuracy of apple prediction is: 0.9948990298435092
15/15 [==============================] - 1s 6ms/step - loss: 0.0077
Validation accuracy of google prediction is: 0.99230588786304
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 2895.41
RMSE_ERROR: 53.81
MAPE_ERROR: 43.39
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 4024.51
RMSE_ERROR: 63.44
MAPE_ERROR: 51.69


In [ ]:
pred('lstm','adagrad','mse',0.0)

*** APPLE ***
Model: "model_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_45 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_122 (Dropout)       (None, 30, 200)           0         
                                                                 
 lstm_46 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_123 (Dropout)       (None, 30, 200)           0         
                                                                 
 lstm_47 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_124 (Dropout)       (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adagrad.py:74: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 5s 146ms/step - loss: 0.0819 - val_loss: 0.0737
Epoch 2/80
9/9 [==============================] - 0s 21ms/step - loss: 0.0607 - val_loss: 0.0562
Epoch 3/80
9/9 [==============================] - 0s 17ms/step - loss: 0.0476 - val_loss: 0.0446
Epoch 4/80
9/9 [==============================] - 0s 18ms/step - loss: 0.0390 - val_loss: 0.0365
Epoch 5/80
9/9 [==============================] - 0s 17ms/step - loss: 0.0330 - val_loss: 0.0307
Epoch 6/80
9/9 [==============================] - 0s 17ms/step - loss: 0.0288 - val_loss: 0.0265
Epoch 7/80
9/9 [==============================] - 0s 16ms/step - loss: 0.0258 - val_loss: 0.0234
Epoch 8/80
9/9 [==============================] - 0s 16ms/step - loss: 0.0237 - val_loss: 0.0211
Epoch 9/80
9/9 [==============================] - 0s 17ms/step - loss: 0.0221 - val_loss: 0.0194
Epoch 10/80
9/9 [==============================] - 0s 17ms/step - loss: 0.0210 - val_loss: 0.0181
Epoch 11/80
9/9 [======================

*** GOOGLE ***
Model: "model_42"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_48 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_125 (Dropout)       (None, 30, 200)           0         
                                                                 
 lstm_49 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_126 (Dropout)       (None, 30, 200)           0         
                                                                 
 lstm_50 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_127 (Dropout)       (None, 100)   

36/36 [==============================] - 0s 5ms/step - loss: 0.0124
accuracy of apple prediction is: 0.987565896473825
37/37 [==============================] - 0s 5ms/step - loss: 0.0067
accuracy of google prediction is: 0.9932676893658936
16/16 [==============================] - 0s 5ms/step - loss: 0.0095
Validation accuracy of apple prediction is: 0.9905420374125242
15/15 [==============================] - 1s 6ms/step - loss: 0.0056
Validation accuracy of google prediction is: 0.9944032281637192
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 5319.73
RMSE_ERROR: 72.94
MAPE_ERROR: 70.62
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 3020.06
RMSE_ERROR: 54.96
MAPE_ERROR: 41.46


In [ ]:
pred('rnn','adagrad','mse',0.0)

*** APPLE ***
Model: "model_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_32 (SimpleRNN)   (None, 30, 200)           40400     
                                                                 
 dropout_128 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_33 (SimpleRNN)   (None, 30, 200)           80200     
                                                                 
 dropout_129 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_34 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_130 (Dropout)       (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adagrad.py:74: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 3s 137ms/step - loss: 0.4192 - val_loss: 0.0048
Epoch 2/80
9/9 [==============================] - 1s 97ms/step - loss: 0.0040 - val_loss: 0.0021
Epoch 3/80
9/9 [==============================] - 1s 97ms/step - loss: 0.0025 - val_loss: 0.0014
Epoch 4/80
9/9 [==============================] - 1s 95ms/step - loss: 0.0018 - val_loss: 0.0012
Epoch 5/80
9/9 [==============================] - 1s 101ms/step - loss: 0.0014 - val_loss: 9.2502e-04
Epoch 6/80
9/9 [==============================] - 1s 89ms/step - loss: 0.0011 - val_loss: 8.1630e-04
Epoch 7/80
9/9 [==============================] - 1s 93ms/step - loss: 9.0977e-04 - val_loss: 6.9704e-04
Epoch 8/80
9/9 [==============================] - 1s 95ms/step - loss: 8.3654e-04 - val_loss: 6.1003e-04
Epoch 9/80
9/9 [==============================] - 1s 100ms/step - loss: 7.0603e-04 - val_loss: 5.8318e-04
Epoch 10/80
9/9 [==============================] - 1s 95ms/step - loss: 6.7010e-04 - val_loss: 5.9210e-

*** GOOGLE ***
Model: "model_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_35 (SimpleRNN)   (None, 30, 200)           40400     
                                                                 
 dropout_131 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_36 (SimpleRNN)   (None, 30, 200)           80200     
                                                                 
 dropout_132 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_37 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_133 (Dropout)       (None, 100)   

36/36 [==============================] - 0s 10ms/step - loss: 3.8602e-04
accuracy of apple prediction is: 0.9996139778813813
37/37 [==============================] - 0s 9ms/step - loss: 5.5668e-04
accuracy of google prediction is: 0.9994433205574751
16/16 [==============================] - 0s 9ms/step - loss: 4.3968e-04
Validation accuracy of apple prediction is: 0.9995603219140321
15/15 [==============================] - 0s 10ms/step - loss: 3.8607e-04
Validation accuracy of google prediction is: 0.9996139280556235
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 621.98
RMSE_ERROR: 24.94
MAPE_ERROR: 13.02
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 912.47
RMSE_ERROR: 30.21
MAPE_ERROR: 17.08


dropout

In [ ]:
pred('gru','adam','mse',0.2)

*** APPLE ***
Model: "model_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_45 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_134 (Dropout)       (None, 30, 200)           0         
                                                                 
 gru_46 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_135 (Dropout)       (None, 30, 200)           0         
                                                                 
 gru_47 (GRU)                (None, 100)               90600     
                                                                 
 dropout_136 (Dropout)       (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 6s 139ms/step - loss: 0.0157 - val_loss: 0.0053
Epoch 2/80
9/9 [==============================] - 0s 23ms/step - loss: 0.0027 - val_loss: 0.0014
Epoch 3/80
9/9 [==============================] - 0s 21ms/step - loss: 0.0013 - val_loss: 5.3324e-04
Epoch 4/80
9/9 [==============================] - 0s 16ms/step - loss: 8.2890e-04 - val_loss: 5.0452e-04
Epoch 5/80
9/9 [==============================] - 0s 16ms/step - loss: 7.8702e-04 - val_loss: 1.8130e-04
Epoch 6/80
9/9 [==============================] - 0s 16ms/step - loss: 6.1633e-04 - val_loss: 1.6105e-04
Epoch 7/80
9/9 [==============================] - 0s 16ms/step - loss: 5.6468e-04 - val_loss: 1.6346e-04
Epoch 8/80
9/9 [==============================] - 0s 17ms/step - loss: 4.6421e-04 - val_loss: 1.4549e-04
Epoch 9/80
9/9 [==============================] - 0s 16ms/step - loss: 4.9662e-04 - val_loss: 1.3286e-04
Epoch 10/80
9/9 [==============================] - 0s 17ms/step - loss: 5.0915e-04 - 

*** GOOGLE ***
Model: "model_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 gru_48 (GRU)                (None, 30, 200)           121800    
                                                                 
 dropout_137 (Dropout)       (None, 30, 200)           0         
                                                                 
 gru_49 (GRU)                (None, 30, 200)           241200    
                                                                 
 dropout_138 (Dropout)       (None, 30, 200)           0         
                                                                 
 gru_50 (GRU)                (None, 100)               90600     
                                                                 
 dropout_139 (Dropout)       (None, 100)   

36/36 [==============================] - 0s 5ms/step - loss: 7.3957e-05
accuracy of apple prediction is: 0.9999260434124153
37/37 [==============================] - 0s 5ms/step - loss: 9.5093e-05
accuracy of google prediction is: 0.9999049068574095
16/16 [==============================] - 0s 5ms/step - loss: 1.0055e-04
Validation accuracy of apple prediction is: 0.999899448492215
15/15 [==============================] - 1s 5ms/step - loss: 5.1568e-05
Validation accuracy of google prediction is: 0.999948432414385
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 12.60
RMSE_ERROR: 3.55
MAPE_ERROR: 1.43
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 15.31
RMSE_ERROR: 3.91
MAPE_ERROR: 1.66


In [ ]:
pred('lstm','rmsprop','mse',0.2)

*** APPLE ***
Model: "model_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_51 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_140 (Dropout)       (None, 30, 200)           0         
                                                                 
 lstm_52 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_141 (Dropout)       (None, 30, 200)           0         
                                                                 
 lstm_53 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_142 (Dropout)       (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 5s 143ms/step - loss: 0.0498 - val_loss: 0.0224
Epoch 2/80
9/9 [==============================] - 0s 21ms/step - loss: 0.0099 - val_loss: 0.0027
Epoch 3/80
9/9 [==============================] - 0s 18ms/step - loss: 0.0019 - val_loss: 7.8830e-04
Epoch 4/80
9/9 [==============================] - 0s 17ms/step - loss: 8.0452e-04 - val_loss: 4.0560e-04
Epoch 5/80
9/9 [==============================] - 0s 18ms/step - loss: 6.2318e-04 - val_loss: 2.6960e-04
Epoch 6/80
9/9 [==============================] - 0s 18ms/step - loss: 6.2720e-04 - val_loss: 4.0048e-04
Epoch 7/80
9/9 [==============================] - 0s 17ms/step - loss: 7.0472e-04 - val_loss: 7.5751e-04
Epoch 8/80
9/9 [==============================] - 0s 16ms/step - loss: 6.0431e-04 - val_loss: 3.1570e-04
Epoch 9/80
9/9 [==============================] - 0s 17ms/step - loss: 6.4837e-04 - val_loss: 5.4369e-04
Epoch 10/80
9/9 [==============================] - 0s 17ms/step - loss: 6.6189e-04 - 

*** GOOGLE ***
Model: "model_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 lstm_54 (LSTM)              (None, 30, 200)           161600    
                                                                 
 dropout_143 (Dropout)       (None, 30, 200)           0         
                                                                 
 lstm_55 (LSTM)              (None, 30, 200)           320800    
                                                                 
 dropout_144 (Dropout)       (None, 30, 200)           0         
                                                                 
 lstm_56 (LSTM)              (None, 100)               120400    
                                                                 
 dropout_145 (Dropout)       (None, 100)   

36/36 [==============================] - 0s 5ms/step - loss: 1.6289e-04
accuracy of apple prediction is: 0.9998371051478898
37/37 [==============================] - 0s 5ms/step - loss: 1.7919e-04
accuracy of google prediction is: 0.9998208090401022
16/16 [==============================] - 0s 5ms/step - loss: 2.2065e-04
Validation accuracy of apple prediction is: 0.9997793541988358
15/15 [==============================] - 1s 6ms/step - loss: 1.2046e-04
Validation accuracy of google prediction is: 0.9998795419596718
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 19.72
RMSE_ERROR: 4.44
MAPE_ERROR: 1.91
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 45.11
RMSE_ERROR: 6.72
MAPE_ERROR: 2.88


In [ ]:
pred('rnn','adam','mse',0.2)

*** APPLE ***
Model: "model_49"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_38 (SimpleRNN)   (None, 30, 200)           40400     
                                                                 
 dropout_146 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_39 (SimpleRNN)   (None, 30, 200)           80200     
                                                                 
 dropout_147 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_40 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_148 (Dropout)       (None, 100)    

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



9/9 [==============================] - 3s 134ms/step - loss: 1.0452 - val_loss: 0.0061
Epoch 2/80
9/9 [==============================] - 1s 92ms/step - loss: 0.3830 - val_loss: 0.0075
Epoch 3/80
9/9 [==============================] - 1s 94ms/step - loss: 0.2286 - val_loss: 0.0234
Epoch 4/80
9/9 [==============================] - 1s 92ms/step - loss: 0.1699 - val_loss: 0.0037
Epoch 5/80
9/9 [==============================] - 1s 94ms/step - loss: 0.1103 - val_loss: 0.0056
Epoch 6/80
9/9 [==============================] - 1s 93ms/step - loss: 0.0962 - val_loss: 0.0175
Epoch 7/80
9/9 [==============================] - 1s 91ms/step - loss: 0.0716 - val_loss: 0.0013
Epoch 8/80
9/9 [==============================] - 1s 100ms/step - loss: 0.0578 - val_loss: 0.0036
Epoch 9/80
9/9 [==============================] - 1s 100ms/step - loss: 0.0433 - val_loss: 0.0033
Epoch 10/80
9/9 [==============================] - 1s 91ms/step - loss: 0.0435 - val_loss: 0.0011
Epoch 11/80
9/9 [====================

*** GOOGLE ***
Model: "model_50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 30, 1)]           0         
                                                                 
 simple_rnn_41 (SimpleRNN)   (None, 30, 200)           40400     
                                                                 
 dropout_149 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_42 (SimpleRNN)   (None, 30, 200)           80200     
                                                                 
 dropout_150 (Dropout)       (None, 30, 200)           0         
                                                                 
 simple_rnn_43 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dropout_151 (Dropout)       (None, 100)   

36/36 [==============================] - 0s 10ms/step - loss: 2.5811e-04
accuracy of apple prediction is: 0.9997418913408183
37/37 [==============================] - 0s 10ms/step - loss: 2.1745e-04
accuracy of google prediction is: 0.9997825480968459
16/16 [==============================] - 0s 10ms/step - loss: 2.7969e-04
Validation accuracy of apple prediction is: 0.9997203095408622
15/15 [==============================] - 0s 10ms/step - loss: 1.6958e-04
Validation accuracy of google prediction is: 0.9998304219334386
**** APPLE ERROR SUMMARY ***
MSE_ERROR: 154.14
RMSE_ERROR: 12.42
MAPE_ERROR: 6.56
**** GOOGLE ERROR SUMMARY ***
MSE_ERROR: 34.01
RMSE_ERROR: 5.83
MAPE_ERROR: 2.42
